In [86]:
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
from torch import optim
import torch.nn.functional as F
import numpy as np

batch_size = 1

In [87]:
# Generate dummy data
input_data = torch.rand((100, 30, 40))
output_data = torch.rand((100, 20, 10))

In [88]:
# Data loader
class Loader(Dataset):
    
    def __len__(self):
        return input_data.size()[0]
    
    def __getitem__(self, idx):  # idx is index for subset of batch data.
        input_frame = input_data[idx]
        output_words = output_data[idx]
        return input_frame, output_words

In [89]:
class EncoderLSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = 1
        self.Lstm = nn.LSTM(input_size = self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True)
    
    def forward(self, input_state, hidden_state):
        output, hidden = self.Lstm(input_state, hidden_state)
        return output, hidden
    
    def initHidden(self):
        return (torch.zeros(1,batch_size,self.hidden_size), torch.zeros(1,batch_size,self.hidden_size))
        
class DecoderLSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = 1
        self.Lstm = nn.LSTM(input_size = self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True)
    
    def forward(self, input_state, hidden_state):

        output, hidden = self.Lstm(input_state, hidden_state)
        return output, hidden
    

In [130]:
class AttnDecoderLSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, enc_time_step, enc_output_size):
        super(AttnDecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = 1
        self.enc_time_step = enc_time_step
        self.enc_output_size = enc_output_size
        self.attn_weights = nn.Linear(self.hidden_size+self.enc_output_size,1)
        self.Lstm = nn.LSTM(input_size = self.input_size+self.enc_output_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True)
        
    def forward(self, input_state, hidden, encoder_output):
        hidden_expand = hidden[0].expand(1, self.enc_time_step, self.hidden_size)
        
        attn_input = torch.cat((hidden_expand, encoder_output),dim=2)
        attn_weights = self.attn_weights(attn_input)
        attn_weights = attn_weights.view(1,1,-1)
        attn_weights = F.softmax(attn_weights, dim=2)
        attn = torch.bmm(attn_weights, encoder_output)
        
        output, hidden = self.Lstm(torch.cat((input_state, attn), dim=2), hidden)
        
        return output, hidden, attn
        

In [131]:
loader = Loader()
dataloader = DataLoader(loader, batch_size=batch_size,shuffle=True)

enc_input_size = 40
dec_input_size = 10
enc_time_step = input_data.size(1)
hidden_size = 128
enc_output_size = hidden_size

for idx, sample in enumerate(dataloader):
    print(idx)
    a, b = sample[0], sample[1]
    print(a.size(), b.size())
    encoder = EncoderLSTM(enc_input_size, hidden_size)
    encoder_hidden = encoder.initHidden()
    encoder_output, encoder_hidden = encoder(a, encoder_hidden)
    
    print('encoder output', encoder_output.size())
    
    decoder_hidden = encoder_hidden
    
    for i in range(20): # decoder time step. in actual it should be <EOS>
        print(i)
        decoder = AttnDecoderLSTM(dec_input_size, hidden_size, enc_time_step, enc_output_size)
        decoder_output, decoder_hidden,_ = decoder(b[0][i].view(1, 1,-1), decoder_hidden, encoder_output)

    break
    
   

0
torch.Size([1, 30, 40]) torch.Size([1, 20, 10])
encoder output torch.Size([1, 30, 128])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(2)

######################################################################

word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"], word_to_ix["world"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)